In [1]:
### Packages

using Printf
using LinearAlgebra
using JLD
using QuadGK
using Statistics
using PyCall
using PyPlot
using LaTeXStrings
using GadgetIO
using GadgetUnits
using GadgetGalaxies
using Unitful
using UnitfulAstro
using Missings
using HypothesisTests
using Distributions
using CSV
using DataFrames
using Cosmology
using Suppressor


### Functions

# Tranform Cartesian vector into [r,θ,ϕ] Spherical vector
function cartesian_to_spherical(x)
    s = zeros(3)
    s[1]    = norm(x) # Radius
    s[2]    = acosd(x[3] / s[1])# * 180 / π # θ[°]
    s[3]    = acosd(x[1] / sqrt(x[1]*x[1] + x[2]*x[2]))# * 180 / π # ϕ[°]
    return s
end

# Rotation Matrix to align x with reference vector
function aligner(x, ref=[0,0,1])
    x       = x ./ norm(x)
    ref     = ref ./ norm(ref)
    vx      = x × ref
    cx      = transpose(x) * ref
    Vx      = [0 -vx[3] vx[2]; vx[3] 0 -vx[1]; -vx[2] vx[1] 0]
    aligner = I + Vx + (Vx * Vx ./ (1+cx))
    return aligner
end

function orbit_J(subID, centID, pathtofile) # Find central sub maybe using SUBFIND -> centID = FSUB[GRNR[subID+1]+1]
    # Read subhalo features
    head        = read_header(pathtofile)
    smst        = convert_units_physical(read_subfind(pathtofile, "SMST"), :mass, head)
    spos        = convert_units_physical(read_subfind(pathtofile, "SPOS"), :pos, head)
    svel        = read_subfind(pathtofile, "SVEL") # no conversion since already physical units??
    # Return with masses added
    return ((spos[:,subID+1] .- spos[:,centID+1]) × (svel[:,subID+1] .- svel[:,centID+1])) .* (smst[1,subID+1]+smst[2,subID+1]+smst[5,subID+1])
end

orbit_J (generic function with 1 method)

In [3]:
### Settings

haloID      = 2
box         = "/HydroSims/Magneticum/Box4/uhr_test"
input_dir   = "/home/moon/sfortune/spinevo/halostories_update_stars"



# Read the halo story

storyfilelist   = readdir(input_dir)
halo_filestring = " "
for i in 1:length(storyfilelist)
    if occursin("halo_$(haloID)_", storyfilelist[i])
        halo_filestring = storyfilelist[i]
    end
end
@show halo_filestring
merger_data = load(joinpath(input_dir, halo_filestring), "merger_data")
halo_story  = load(joinpath(input_dir, halo_filestring), "halo_story")

halo_filestring = "halo_2_2323.jld"


Dict{String, Any} with 26 entries:
  "BOX"        => "/HydroSims/Magneticum/Box4/uhr_test/"
  "M_GAS"      => [691.555, 649.392, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  … …
  "I_SUB"      => [2323, 2198, 2555, 2554, 2542, 2532, 2503, 2500, 2479, 2464  …
  "J_STARS"    => Union{Missing, Float64}[2.08101e14 missing … missing -7.32577…
  "J_GAS"      => Union{Missing, Float64}[2.34737e14 missing … missing missing;…
  "SNAP"       => [136, 135, 135, 135, 135, 135, 135, 135, 135, 135  …  80, 79,…
  "M_DM"       => [7363.46, 7137.17, 0.0145457, 0.0, 0.0145457, 0.0509099, 0.0,…
  "RHMS_STARS" => Union{Missing, Float64}[20.4076, missing, missing, missing, m…
  "I_TREE"     => [291866, 291867, 2103756, 2080827, 2107070, 2085213, 2100469,…
  "REDSHIFT"   => [0.0663401, 0.0749363, 0.0749363, 0.0749363, 0.0749363, 0.074…
  "J_DM"       => Union{Missing, Float64}[1.57877e18 missing … missing missing;…
  "RHMS_GAS"   => Union{Missing, Float64}[136.495, missing, missing, missing, m…
  "M_STARS"    =>

In [6]:
# Playground

println(size((halo_story["j_STARS"])))



(3, 169)


In [ ]:
# Identify First Progenitors and mergers

loop_nmergers   = 0
loop_mmergers   = 0.
loop_J_sum      = zeros(3)
mass_missed     = 0.
mass_added      = 0.

fp_indices      = Array{Int64}(undef, 0)
fp_snaps        = Array{Int64}(undef, 0)
n_mergers       = Array{Int64}(undef, 0)
merger_indices  = Array{Int64}(undef, 0)

loop_length     = length(halo_story["SNAP"])
# First Progenitors, forward in time by starting at the bottom
    
for i in loop_length:-1:1
    if i==1 || halo_story["SNAP"][i-1] > halo_story["SNAP"][i] && !ismissing(halo_story["J_STARS"][1,i]) # First Progenitor
        #print("FP@SNAP$(halo_story["SNAP"][i]) ")
        #flush(stdout)

        fp_indices  = vcat( fp_indices, i )
        fp_snaps    = vcat( fp_snaps, halo_story["SNAP"][i] )
    end
end


merger_count    = 0
# check if first snap already contains a first progenitor
if fp_snaps[1] == halo_story["SNAP"][end]
    fp_index    = 2
    n_mergers   = vcat( n_mergers, merger_count )
else
    fp_index    = 1
end
# Merger Map, forward in time by starting at the bottom
 for i in loop_length:-1:1
    # Check out Merger Info and assign to next
    if halo_story["SNAP"][i] == fp_snaps[fp_index] # First Progenitor
        n_mergers   = vcat( n_mergers, merger_count )
        merger_count = 0
        fp_index   += 1
    end

    # Identify Mergers
    if i == 1 || halo_story["SNAP"][i] < halo_story["SNAP"][i-1]
        print(" ")
    elseif halo_story["SNAP"][i] == halo_story["SNAP"][i-1] # Merger
        merger_count   += 1
        merger_indices  = vcat( merger_indices, i )
    else
        println("Error for i = $i, $(halo_story["SNAP"][i]), $(fp_snaps[fp_index]), $(halo_story["SNAP"][i-1]), $(halo_story["I_SUB"][i-1])")
    end
end

println("$(length(fp_indices)) $(length(fp_snaps)) $(length(n_mergers))")
println("$(length(merger_indices)) $(sum(n_mergers)) ")

                                                             18 18 18
108 108 


In [5]:
# Accessing specific timestep and result

test = 10
println("$(fp_indices[test])")
println("$(length(merger_indices[1+sum(n_mergers[1:test-1]):sum(n_mergers[1:test])]))")
@show merger_indices[1+sum(n_mergers[1:test-1]):sum(n_mergers[1:test])]

98
1
merger_indices[1 + sum(n_mergers[1:test - 1]):sum(n_mergers[1:test])] = [104]


1-element Vector{Int64}:
 104

In [7]:
# Fill the dictionary

merger_collection_STARS = Dict(
        "SNAP"          => Array{Int64}(undef, 0),
        "REDSHIFT"      => Array{Float64}(undef, 0),
        "LOOKBACKTIME"  => Array{Float64}(undef, 0),
        "I_SUB"         => Array{Int64}(undef, 0),
        "M_MM"     => Array{Float64}(undef, 0),
        "J_MMorbital"      => Array{Float64}(undef, 3, 0), 
        "J_SUMorbital"         => Array{Float64}(undef, 3, 0), 
        "δM_felix"       => Array{Float64}(undef, 0), 
        "δM_fromJ"       => Array{Float64}(undef, 0), 
        "M_felix"       => Array{Float64}(undef, 0), 
        "M_fromJ"       => Array{Float64}(undef, 0), 
        "ϕ_flip"      => Array{Float64}(undef, 0), 
        "δJ_main"       => Array{Float64}(undef, 3, 0), 
        "N_MERGERS"     => Array{Int64}(undef, 0),
        "M_MERGERS"     => Array{Float64}(undef, 0),
        "M_MISSED"      => Array{Float64}(undef, 0), 
        "M_CONSIDERED"  => Array{Float64}(undef, 0),  
        "M2_MERGERS"     => Array{Float64}(undef, 0),
        "M2_MISSED"      => Array{Float64}(undef, 0), 
        "M2_CONSIDERED"  => Array{Float64}(undef, 0),  
        "BVAL"      => Array{Float64}(undef, 0), 
        "δBVAL"      => Array{Float64}(undef, 0), 
        "J_main"        => Array{Float64}(undef, 3, 0), 
        "j_main"        => Array{Float64}(undef, 3, 0), 
        "δj_main"       => Array{Float64}(undef, 3, 0))

for i in 1:length(fp_indices)
    #print("$i ")
    #flush(stdout)
    
    # Basic Info
    head    = read_header("$box/groups_$(@sprintf("%03i", halo_story["SNAP"][fp_indices[i]]))/sub_$(@sprintf("%03i", halo_story["SNAP"][fp_indices[i]]))")
    merger_collection_STARS["LOOKBACKTIME"] = vcat( merger_collection_STARS["LOOKBACKTIME"], ustrip(lookback_time(cosmology(h=head.h0, OmegaM=head.omega_0), head.z)) )
    merger_collection_STARS["SNAP"]         = vcat( merger_collection_STARS["SNAP"], halo_story["SNAP"][fp_indices[i]] )
    merger_collection_STARS["REDSHIFT"]     = vcat( merger_collection_STARS["REDSHIFT"], halo_story["REDSHIFT"][fp_indices[i]] )
    merger_collection_STARS["I_SUB"]        = vcat( merger_collection_STARS["I_SUB"], halo_story["I_SUB"][fp_indices[i]] )
    merger_collection_STARS["M_felix"]      = vcat( merger_collection_STARS["M_felix"], halo_story["M_STARS"][fp_indices[i]] )
    merger_collection_STARS["M_fromJ"]      = vcat( merger_collection_STARS["M_fromJ"], norm(halo_story["J_STARS"][:,fp_indices[i]]) / norm(halo_story["j_STARS"][:,fp_indices[i]]) )
    merger_collection_STARS["BVAL"]         = vcat( merger_collection_STARS["BVAL"], halo_story["BVAL"][fp_indices[i]] )
    merger_collection_STARS["J_main"]       = hcat( merger_collection_STARS["J_main"], halo_story["J_STARS"][:,fp_indices[i]] )
    merger_collection_STARS["j_main"]       = hcat( merger_collection_STARS["j_main"], halo_story["j_STARS"][:,fp_indices[i]] )
    merger_collection_STARS["N_MERGERS"]    = vcat( merger_collection_STARS["N_MERGERS"], length(merger_indices[1+sum(n_mergers[1:i-1]):sum(n_mergers[1:i])]) )
    
    # Transitional Data
    if i == 1
        merger_collection_STARS["δJ_main"]  = hcat( merger_collection_STARS["δJ_main"], zeros(3) )
        merger_collection_STARS["δj_main"]  = hcat( merger_collection_STARS["δj_main"], zeros(3) )
        merger_collection_STARS["δBVAL"]    = vcat( merger_collection_STARS["δBVAL"], 0 )
        merger_collection_STARS["ϕ_flip"]   = vcat( merger_collection_STARS["ϕ_flip"], 0 )
        merger_collection_STARS["δM_felix"] = vcat( merger_collection_STARS["δM_felix"], 0 )
        merger_collection_STARS["δM_fromJ"] = vcat( merger_collection_STARS["δM_fromJ"], 0. )
        merger_collection_STARS["M_MM"]             = vcat( merger_collection_STARS["M_MM"], 0 )
        merger_collection_STARS["J_MMorbital"]      = hcat( merger_collection_STARS["J_MMorbital"], zeros(3) )
        merger_collection_STARS["J_SUMorbital"]     = hcat( merger_collection_STARS["J_SUMorbital"], zeros(3) )
        merger_collection_STARS["M_MERGERS"]        = vcat( merger_collection_STARS["M_MERGERS"], 0 )
        merger_collection_STARS["M_MISSED"]         = vcat( merger_collection_STARS["M_MISSED"], 0 )
        merger_collection_STARS["M_CONSIDERED"]     = vcat( merger_collection_STARS["M_CONSIDERED"], 0 )
        merger_collection_STARS["M2_MERGERS"]       = vcat( merger_collection_STARS["M2_MERGERS"], 0 )
        merger_collection_STARS["M2_MISSED"]        = vcat( merger_collection_STARS["M2_MISSED"], 0 )
        merger_collection_STARS["M2_CONSIDERED"]    = vcat( merger_collection_STARS["M2_CONSIDERED"], 0 )
    else
        merger_collection_STARS["δJ_main"]  = hcat( merger_collection_STARS["δJ_main"], halo_story["J_STARS"][:,fp_indices[i]] - halo_story["J_STARS"][:,fp_indices[i-1]] )
        merger_collection_STARS["δj_main"]  = hcat( merger_collection_STARS["δj_main"], halo_story["j_STARS"][:,fp_indices[i]] - halo_story["j_STARS"][:,fp_indices[i-1]] )
        merger_collection_STARS["δBVAL"]    = vcat( merger_collection_STARS["δBVAL"], halo_story["BVAL"][fp_indices[i]] - halo_story["BVAL"][fp_indices[i-1]] )
        if !ismissing(halo_story["j_STARS"][1,fp_indices[i]]) && !ismissing(halo_story["j_STARS"][2,fp_indices[i]]) && !ismissing(halo_story["j_STARS"][3,fp_indices[i]])
            merger_collection_STARS["ϕ_flip"]   = vcat( merger_collection_STARS["ϕ_flip"], acosd( transpose(halo_story["J_STARS"][:,fp_indices[i]]) * halo_story["J_STARS"][:,fp_indices[i-1]] / norm(halo_story["J_STARS"][:,fp_indices[i]]) / norm(halo_story["J_STARS"][:,fp_indices[i-1]]) ) )
        else
            println(halo_story["j_STARS"][:,fp_indices[i]])
        end
        #println("\n$(halo_story["J_STARS"][:,fp_indices[i]]) $(halo_story["J_STARS"][:,fp_indices[i-1]])")
        merger_collection_STARS["δM_felix"] = vcat( merger_collection_STARS["δM_felix"], halo_story["M_STARS"][fp_indices[i]]-halo_story["M_STARS"][fp_indices[i-1]] )
        merger_collection_STARS["δM_fromJ"] = vcat( merger_collection_STARS["δM_fromJ"], (norm(halo_story["J_STARS"][:,fp_indices[i]]) / norm(halo_story["j_STARS"][:,fp_indices[i]])) - (norm(halo_story["J_STARS"][:,fp_indices[i-1]]) / norm(halo_story["j_STARS"][:,fp_indices[i-1]])) )
        
        #Merger Data
        # Setup
        merger_collection_STARS["M_MM"]             = vcat( merger_collection_STARS["M_MM"], 0 )
        merger_collection_STARS["J_MMorbital"]      = hcat( merger_collection_STARS["J_MMorbital"], zeros(3) )
        merger_collection_STARS["J_SUMorbital"]     = hcat( merger_collection_STARS["J_SUMorbital"], zeros(3) )
        merger_collection_STARS["M_MERGERS"]        = vcat( merger_collection_STARS["M_MERGERS"], 0 )
        merger_collection_STARS["M_MISSED"]         = vcat( merger_collection_STARS["M_MISSED"], 0 )
        merger_collection_STARS["M_CONSIDERED"]     = vcat( merger_collection_STARS["M_CONSIDERED"], 0 )
        merger_collection_STARS["M2_MERGERS"]       = vcat( merger_collection_STARS["M2_MERGERS"], 0 )
        merger_collection_STARS["M2_MISSED"]        = vcat( merger_collection_STARS["M2_MISSED"], 0 )
        merger_collection_STARS["M2_CONSIDERED"]    = vcat( merger_collection_STARS["M2_CONSIDERED"], 0 )
        # Actual check
        for ii in merger_indices[1+sum(n_mergers[1:i-1]):sum(n_mergers[1:i])]
            # Most Massive condition
            head    = read_header("$box/groups_$(@sprintf("%03i", halo_story["SNAP"][ii]))/sub_$(@sprintf("%03i", halo_story["SNAP"][ii]))")
            if convert_units_physical_mass(halo_story["M_STAR_2"][ii], head) > merger_collection_STARS["M_MM"][end]
                merger_collection_STARS["M_MM"][end]            = convert_units_physical_mass(halo_story["M_STAR_2"][ii], head)
                merger_collection_STARS["J_MMorbital"][:,end]   = halo_story["J_orbital"][:,ii]
            end
            # Orbital Data
            if !ismissing(halo_story["J_orbital"][1,ii]) || !ismissing(halo_story["J_orbital"][2,ii]) || !ismissing(halo_story["J_orbital"][3,ii])
                merger_collection_STARS["J_SUMorbital"][:,end] .+= halo_story["J_orbital"][:,ii]
                merger_collection_STARS["M_MERGERS"][end]       += convert_units_physical_mass(halo_story["M_STARS"][ii], head)
                merger_collection_STARS["M_CONSIDERED"][end]    += convert_units_physical_mass(halo_story["M_STARS"][ii], head)
                merger_collection_STARS["M2_MERGERS"][end]      += convert_units_physical_mass(halo_story["M_STAR_2"][ii], head)
                merger_collection_STARS["M2_CONSIDERED"][end]   += convert_units_physical_mass(halo_story["M_STAR_2"][ii], head)
                println(halo_story["J_orbital"][:,ii])
            else
                println("$ii")
                merger_collection_STARS["M_MERGERS"][end]   += convert_units_physical_mass(halo_story["M_STARS"][ii], head)
                merger_collection_STARS["M_MISSED"][end]    += convert_units_physical_mass(halo_story["M_STARS"][ii], head)
                merger_collection_STARS["M2_MERGERS"][end]  += convert_units_physical_mass(halo_story["M_STAR_2"][ii], head)
                merger_collection_STARS["M2_MISSED"][end]   += convert_units_physical_mass(halo_story["M_STAR_2"][ii], head)
            end
        end
    end
end
# Sanity check
println("$(size(merger_collection_STARS["SNAP"]))")
println("$(size(merger_collection_STARS["δJ_main"]))")
println("$(size(merger_collection_STARS["J_SUMorbital"]))")
println("$(size(merger_collection_STARS["M2_MERGERS"]))")
@show merger_collection_STARS["M2_MISSED"]

Union{Missing, Float32}[1.5795273f13, -1.7181782f13, -1.9293167f13]
Union{Missing, Float32}[-4.226302f11, 9.299851f12, -1.5297084f11]
Union{Missing, Float32}[4.854231f12, -4.852555f13, -4.452f12]
Union{Missing, Float32}[2.2165198f12, 5.1844093f12, -5.8868965f12]
Union{Missing, Float32}[-2.5403297f13, 2.8167297f13, 4.0564506f13]
Union{Missing, Float32}[1.4301139f13, -6.6326784f12, -1.7580683f12]
Union{Missing, Float32}[-9.178594f12, -4.600427f13, 1.1102914f13]
Union{Missing, Float32}[2.2486708f13, 2.2184754f13, -9.365616f13]
Union{Missing, Float32}[4.0888097f13, -2.9188851f13, -2.0822886f12]
Union{Missing, Float32}[1.6231618f14, 1.1090422f14, -1.4147552f14]
Union{Missing, Float32}[1.07436f13, 1.1066285f13, -9.977059f12]
Union{Missing, Float32}[1.5437116f13, -5.90714f11, -1.3753318f12]
Union{Missing, Float32}[2.309475f13, -1.2612334f13, -5.476687f12]
Union{Missing, Float32}[-3.806283f12, 4.1271029f12, -4.2467894f12]
Union{Missing, Float32}[-1.0684899f13, -8.2783465f13, 4.6141135f12]
Unio

18-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0